<a href="https://colab.research.google.com/github/nineMAN9/mh429-is465-/blob/master/classifydocuments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "Document Classification.ipynb",
      "provenance": [],
      "include_colab_link": true
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "view-in-github",
        "colab_type": "text"
      },
      "source": [
        "<a href=\"https://github.com/nineMAN9/mh429-is465-/blob/master/classifydocuments" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "RJ9M9H7cG99X",
        "colab_type": "code",
        "outputId": "dd3ec626-8bf9-4b7e-cc02-c60c27ce9cab",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 33
        }
      },
      "source": [
        "print('hello world')"
      ],
      "execution_count": 0,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "hello world\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "uyK9R3aSOrDQ",
        "colab_type": "text"
      },
      "source": [
        "Start"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "2kprkhf1HhZA",
        "colab_type": "code",
        "colab": {}
      },
      "source": [
        "\n",
        "\n",
        "# Author: Olivier Grisel <olivier.grisel@ensta.org>\n",
        "#         Lars Buitinck\n",
        "#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>\n",
        "# License: BSD 3 clause\n"
      ],
      "execution_count": 0,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "UX307rXJHrTD",
        "colab_type": "text"
      },
      "source": [
        "import some packages: time is python time \n",
        "\n",
        "web mining is mining the text\n",
        "web mining is fun\n",
        "I like this class\n",
        "idf log3/2\n",
        "tf-idf 1/6*log3/2\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "Ye90Rd_JIYm2",
        "colab_type": "code",
        "colab": {}
      },
      "source": [
        "from time import time\n",
        "\n",
        "from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer\n",
        "from sklearn.decomposition import NMF, LatentDirichletAllocation\n",
        "from sklearn.datasets import fetch_20newsgroups"
      ],
      "execution_count": 0,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "rU0xULmiKKIq",
        "colab_type": "text"
      },
      "source": [
        "Define some variables "
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "DLJv0YT4KOxX",
        "colab_type": "code",
        "outputId": "6a9c8b0a-f8df-4209-be22-548352fd614c",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 33
        }
      },
      "source": [
        "n_samples = 2000\n",
        "n_features = 1000\n",
        "n_components = 10\n",
        "n_top_words = 20\n",
        "print(n_samples)"
      ],
      "execution_count": 0,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "2000\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "kGm4eNacKk4J",
        "colab_type": "text"
      },
      "source": [
        "Define function"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "bFPFisMbKmvA",
        "colab_type": "code",
        "colab": {}
      },
      "source": [
        "def print_top_words(model, feature_names, n_top_words):\n",
        "    for topic_idx, topic in enumerate(model.components_):\n",
        "        message = \"Topic #%d: \" % topic_idx\n",
        "        message += \" \".join([feature_names[i]\n",
        "                             for i in topic.argsort()[:-n_top_words - 1:-1]])\n",
        "        print(message)\n",
        "    print()"
      ],
      "execution_count": 0,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "nCOi9Gc9K9QU",
        "colab_type": "text"
      },
      "source": [
        "# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics\n",
        "# to filter out useless terms early on: the posts are stripped of headers,\n",
        "# footers and quoted replies, and common English words, words occurring in\n",
        "# only one document or in at least 95% of the documents are removed."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "PWjs_4u4LOT7",
        "colab_type": "code",
        "outputId": "360229a2-a867-4225-c357-b79f4fff885a",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 50
        }
      },
      "source": [
        "print(\"Loading dataset...\")\n",
        "t0 = time()\n",
        "data, _ = fetch_20newsgroups(shuffle=True, random_state=1,\n",
        "                             remove=('headers', 'footers', 'quotes'),\n",
        "                             return_X_y=True)\n",
        "data_samples = data[:n_samples]\n",
        "print(\"done in %0.3fs.\" % (time() - t0))"
      ],
      "execution_count": 0,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "Loading dataset...\n",
            "done in 1.444s.\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "3TB5ty8PLsFg",
        "colab_type": "text"
      },
      "source": [
        "# Use tf-idf features for NMF."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "wNryVPxrLxYv",
        "colab_type": "code",
        "outputId": "a7d876b2-cbbb-4d15-bd25-510dff76a35f",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 50
        }
      },
      "source": [
        "print(\"Extracting tf-idf features for NMF...\")\n",
        "tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,\n",
        "                                   max_features=n_features,\n",
        "                                   stop_words='english')\n",
        "t0 = time()\n",
        "tfidf = tfidf_vectorizer.fit_transform(data_samples)\n",
        "print(\"done in %0.3fs.\" % (time() - t0))"
      ],
      "execution_count": 0,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "Extracting tf-idf features for NMF...\n",
            "done in 0.363s.\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "-gUH-fM-MDL_",
        "colab_type": "text"
      },
      "source": [
        "# Use tf (raw term count) features for LDA."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "iUMvwNYQMG-I",
        "colab_type": "code",
        "outputId": "ec88bc21-2efa-4a8c-f5af-a151d4852b11",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 67
        }
      },
      "source": [
        "print(\"Extracting tf features for LDA...\")\n",
        "tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,\n",
        "                                max_features=n_features,\n",
        "                                stop_words='english')\n",
        "t0 = time()\n",
        "tf = tf_vectorizer.fit_transform(data_samples)\n",
        "print(\"done in %0.3fs.\" % (time() - t0))\n",
        "print()"
      ],
      "execution_count": 0,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "Extracting tf features for LDA...\n",
            "done in 0.362s.\n",
            "\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "QVGGQs2XMQoD",
        "colab_type": "text"
      },
      "source": [
        "# Fit the NMF model"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "ZYYfShQ-MS3z",
        "colab_type": "code",
        "outputId": "99ecf7ab-320c-4c20-f6bd-33657eb47361",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 268
        }
      },
      "source": [
        "print(\"Fitting the NMF model (Frobenius norm) with tf-idf features, \"\n",
        "      \"n_samples=%d and n_features=%d...\"\n",
        "      % (n_samples, n_features))\n",
        "t0 = time()\n",
        "nmf = NMF(n_components=n_components, random_state=1,\n",
        "          alpha=.1, l1_ratio=.5).fit(tfidf)\n",
        "print(\"done in %0.3fs.\" % (time() - t0))\n",
        "\n",
        "print(\"\\nTopics in NMF model (Frobenius norm):\")\n",
        "tfidf_feature_names = tfidf_vectorizer.get_feature_names()\n",
        "print_top_words(nmf, tfidf_feature_names, n_top_words)"
      ],
      "execution_count": 0,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...\n",
            "done in 0.379s.\n",
            "\n",
            "Topics in NMF model (Frobenius norm):\n",
            "Topic #0: just people don think like know time good make way really say right ve want did ll new use years\n",
            "Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work\n",
            "Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion\n",
            "Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need\n",
            "Topic #4: car cars tires miles 00 new engine insurance price condition oil power speed good 000 brake year models used bought\n",
            "Topic #5: edu soon com send university internet mit ftp mail cc pub article information hope program mac email home contact blood\n",
            "Topic #6: file problem files format win sound ftp pub read save site help image available create copy running memory self version\n",
            "Topic #7: game team games year win play season players nhl runs goal hockey toronto division flyers player defense leafs bad teams\n",
            "Topic #8: drive drives hard disk floppy software card mac computer power scsi controller apple mb 00 pc rom sale problem internal\n",
            "Topic #9: key chip clipper keys encryption government public use secure enforcement phone nsa communications law encrypted security clinton used legal standard\n",
            "\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "i9mcpvOKNT7m",
        "colab_type": "text"
      },
      "source": [
        "# Fit the NMF model"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "4oVsBWEoNXKF",
        "colab_type": "code",
        "outputId": "f9572517-4b0c-43ac-a2c0-874535a54172",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 268
        }
      },
      "source": [
        "print(\"Fitting the NMF model (generalized Kullback-Leibler divergence) with \"\n",
        "      \"tf-idf features, n_samples=%d and n_features=%d...\"\n",
        "      % (n_samples, n_features))\n",
        "t0 = time()\n",
        "nmf = NMF(n_components=n_components, random_state=1,\n",
        "          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,\n",
        "          l1_ratio=.5).fit(tfidf)\n",
        "print(\"done in %0.3fs.\" % (time() - t0))\n",
        "\n",
        "print(\"\\nTopics in NMF model (generalized Kullback-Leibler divergence):\")\n",
        "tfidf_feature_names = tfidf_vectorizer.get_feature_names()\n",
        "print_top_words(nmf, tfidf_feature_names, n_top_words)\n",
        "\n"
      ],
      "execution_count": 0,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=2000 and n_features=1000...\n",
            "done in 1.507s.\n",
            "\n",
            "Topics in NMF model (generalized Kullback-Leibler divergence):\n",
            "Topic #0: people don just like think did say time make know really right said things way ve course didn question probably\n",
            "Topic #1: windows help thanks using hi looking info video dos pc does anybody ftp appreciated mail know advance available use card\n",
            "Topic #2: god does jesus true book christian bible christians religion faith believe life church christ says know read exist lord people\n",
            "Topic #3: thanks know bike interested mail like new car edu heard just price list email hear want cars thing sounds reply\n",
            "Topic #4: 10 00 sale time power 12 new 15 year 30 offer condition 14 16 model 11 monitor 100 old 25\n",
            "Topic #5: space government number public data states earth security water research nasa general 1993 phone information science technology provide blood internet\n",
            "Topic #6: edu file com program soon try window problem remember files sun send library article mike wrong think code win manager\n",
            "Topic #7: game team year games play win season points world division won players nhl flyers toronto case cubs teams ll record\n",
            "Topic #8: drive think hard software disk drives apple computer mac need scsi card don problem read floppy post cable going ii\n",
            "Topic #9: use good just key chip got like ll way clipper doesn keys don better speed stuff want sure going need\n",
            "\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "dzpfrgimOTWe",
        "colab_type": "text"
      },
      "source": [
        "LDA Models"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "fPSzvrP4OUOG",
        "colab_type": "code",
        "outputId": "f619bb3b-6824-4b15-a251-471e2b24ca4e",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 268
        }
      },
      "source": [
        "print(\"Fitting LDA models with tf features, \"\n",
        "      \"n_samples=%d and n_features=%d...\"\n",
        "      % (n_samples, n_features))\n",
        "lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,\n",
        "                                learning_method='online',\n",
        "                                learning_offset=50.,\n",
        "                                random_state=0)\n",
        "t0 = time()\n",
        "lda.fit(tf)\n",
        "print(\"done in %0.3fs.\" % (time() - t0))\n",
        "\n",
        "print(\"\\nTopics in LDA model:\")\n",
        "tf_feature_names = tf_vectorizer.get_feature_names()\n",
        "print_top_words(lda, tf_feature_names, n_top_words)"
      ],
      "execution_count": 0,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "Fitting LDA models with tf features, n_samples=2000 and n_features=1000...\n",
            "done in 4.427s.\n",
            "\n",
            "Topics in LDA model:\n",
            "Topic #0: edu com mail send graphics ftp pub available contact university list faq ca information cs 1993 program sun uk mit\n",
            "Topic #1: don like just know think ve way use right good going make sure ll point got need really time doesn\n",
            "Topic #2: christian think atheism faith pittsburgh new bible radio games alt lot just religion like book read play time subject believe\n",
            "Topic #3: drive disk windows thanks use card drives hard version pc software file using scsi help does new dos controller 16\n",
            "Topic #4: hiv health aids disease april medical care research 1993 light information study national service test led 10 page new drug\n",
            "Topic #5: god people does just good don jesus say israel way life know true fact time law want believe make think\n",
            "Topic #6: 55 10 11 18 15 team game 19 period play 23 12 13 flyers 20 25 22 17 24 16\n",
            "Topic #7: car year just cars new engine like bike good oil insurance better tires 000 thing speed model brake driving performance\n",
            "Topic #8: people said did just didn know time like went think children came come don took years say dead told started\n",
            "Topic #9: key space law government public use encryption earth section security moon probe enforcement keys states lunar military crime surface technology\n",
            "\n"
          ],
          "name": "stdout"
        }
      ]
    }
  ]
} 

SyntaxError: ignored